In [ ]:
from __future__ import print_function
import pickle
import os.path
import json
import os
import requests
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.http import MediaFileUpload

CLIENT_SECRET_FILE ='OAuth_2/google_drive_client_secret.json'
SCOPES = ['https://www.googleapis.com/auth/drive']

def auth():
    creds = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRET_FILE, SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)
    global service, access_token
    service = build('drive', 'v3', credentials=creds)
    access_token = creds.token

def uploadFile():
    print("hello")
    filename = 'data/CA_youtube_trending_data.csv'
    filesize = os.path.getsize(filename)

    # 1. Retrieve session for resumable upload.
    headers = {"Authorization": "Bearer " + access_token, "Content-Type": "application/json"}
    params = {
        "name": "CA_youtube_trending_data.csv",
        "mimeType": "text/csv"
    }
    r = requests.post(
        "https://www.googleapis.com/upload/drive/v3/files?uploadType=resumable",
        headers=headers,
        data=json.dumps(params)
    )
    location = r.headers['Location']


    # 2. Upload the file.
    headers = {"Content-Range": "bytes 0-" + str(filesize - 1) + "/" + str(filesize)}
    r = requests.put(
        location,
        headers=headers,
        data=open(filename, 'rb')
    )
    print(r.text)

auth()
uploadFile()
    